In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# G konstanta
G_konstanta = 6.67e-11  # SI

# parameter model
x0 = 500  # m
z0 = 300  # m
R = 100  # m
rho = 2400  # kg/m3

# variabel bebas x
x = np.arange(0, 1001, 50)

# fungsi forward model bola homogen
def f(x0, z0, R, rho):
    return (
        G_konstanta
        * ((4 / 3) * np.pi * (R ** 3) * z0 * rho)
        / (((x - x0) ** 2 + (z0 ** 2)) ** (3 / 2))
        * 10e5
    )

# matriks kernel
G = np.column_stack([f(x0, z0, R, rho)])

# fungsi bola
def bola(x0, z0, R):
    q = np.linspace(0, 2 * np.pi, 1000)

    xBola = R * np.cos(q) + x0
    zBola = R * np.sin(q) + z0

    return xBola, zBola

g0 = f(x0, z0, R, rho)

# data sintetik
g0 = f(x0, z0, R, rho)  # gobs
xb1, yb1 = bola(x0, z0, R)  # plot bola
xb3, yb3 = bola(x0, z0, R)

# Tambahkan noise
noise_std_dev = 1 # Standar deviasi noise
noise = np.random.normal(0, noise_std_dev, g0.shape)
g0_with_noise = g0 + noise

##inversi non linear ekspansi taylor orde 1
# data sitetik

# model awal
model_awal = [600, 400]  # (x0, z0)

banyak_iterasi = 500
misfit = np.zeros(banyak_iterasi)
misfit_iterasi = []

# Hitung varian dari setiap data observasi
variance = np.var(g0_with_noise)

# Hitung bobot inversi sebagai invers dari varians
weights = 1 / variance

# Iterasi pembobotan
for i in range(banyak_iterasi):

    # forward
    gcal = f(model_awal[0], model_awal[1], R, rho)
    delta_g = g0_with_noise - gcal

    # Menghitung RMS (Root Mean Square) dengan bobot
    rms = np.sqrt(np.mean((delta_g ** 2) * weights))
    # Simpan RMS ke dalam misfit
    misfit_iterasi = np.append(misfit_iterasi, rms)

    # matrix jacobian
    dgz_dx0 = (G_konstanta * (4 / 3)* np.pi* (R ** 3)* model_awal[1]* rho* 2* (x - model_awal[0])/ (((x - model_awal[0]) ** 2 + (0 - model_awal[1]) ** 2) ** (5 / 2))* 10e5)
    dgz_dz0 = (G_konstanta* (4 / 3)* np.pi* (R ** 3)* model_awal[1]* rho* 2* (0 - model_awal[1])/ (((x - model_awal[0]) ** 2 + (0 - model_awal[1]) ** 2) ** (5 / 2))* 10e5)
    J = np.column_stack([dgz_dx0, dgz_dz0])

    # inversi
    # Masukkan bobot dengan variance weighting
    model_cal = np.linalg.inv((J.T).dot(J) + np.diag(weights.flatten())).dot(J.T).dot(delta_g)

    # update model
    model_awal = model_awal + model_cal


xb, yb = bola(x0, z0, R)
xb1, yb1 = bola(model_awal[0], model_awal[1], R)
gcal = f(model_awal[0], model_awal[1], R, rho)

# plot
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(6, 6))

ax1.set_title("Kurva Respon Anomali Gayaberat")
ax1.plot(x, g0, "ko", label="data sintetik")
ax1.plot(x, gcal, "r", label="data kalkulasi")
ax1.set_ylabel("Anomali Gravitasi (mGal)")
ax1.legend()
ax1.grid()

ax2.plot(xb1, yb1, "r")
ax2.plot(xb, yb, "g")
ax2.set_ylim(-50, 700)
ax2.grid()
ax2.set_ylabel("kedalaman (m)")
ax2.set_xlabel("jarak (m)")
ax2.invert_yaxis()

plt.show()

# Plot misfit vs iterasi
plt.figure(figsize=(6, 6))
plt.plot(np.arange(1, banyak_iterasi + 1), misfit_iterasi, 'b')
plt.title('Misfit vs Iterasi')
plt.xlabel('Iterasi')
plt.ylabel('Misfit (RMS)')
plt.grid(True)
plt.show()

print('Nilai RMS adalah :', rms)